[Example Notebook](https://colab.research.google.com/drive/1bPhY1Awz0NIuzGy9RX72eZRB10FwZY0j?usp=sharing#scrollTo=6o7h7-SdMKHK)

In [1]:
import os
import requests
import subprocess
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages 
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from dotenv import load_dotenv, find_dotenv

In [4]:
load_dotenv(find_dotenv())

True

## Normal Execution

In [45]:
prompt = """GENERAL INSTRUCTIONS
You are a domain expert. Your task is to break down a complex question into simpler sub-parts so that it will be 
easier to answer a complex question. Think of it as step by step approach to reach to a solution.

USER QUESTION
{user_question}

ANSWER FORMAT
Provide sub questions in dictionary format with key as "sub-questions" and sub questions in list as value of the dictionary
"""

In [57]:
prompt = """
You are a helpful assistant which answers user's questions. For a complex question which you think
you need some help, then you can request some help from available help or use tool from available tool.

USER QUESTION
{user_question}

AVAIALABLE TOOL
1) Search Tool
2) Math Tool

AVAILABLE HELP
Decomposition: Breaks complex question into simple sub questions to reach to the final answer

Provide final answer in a dictionary format with keys as 'Tool_Request' and 'Helper_Request'

Update the value of the key if you used the help or the tool. If you doesn't use then update it with "Not Required".

NO PREAMBLE NEEDED.
"""

In [61]:
prompt_template = ChatPromptTemplate.from_template(prompt)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.environ["OPENAI_API_KEY"])

chain = prompt_template | llm | StrOutputParser()

In [63]:
json.loads(chain.invoke("how much did the revenue increaases for Apple between Q1 of 2024 and Q2 of 2024?"))

{'Tool_Request': 'Search Tool', 'Helper_Request': 'Not Required'}

## Started with Tools

In [42]:
# first tool -  Calculator

@tool
def calculate(operation):
    """Useful to perform any mathematical calculations,
    like sum, minus, multiplication, division, etc.
    The input to this tool should be a mathematical
    expression, a couple examples are `200*7` or `5000/2*10`
    """
    try:
        return eval(operation)
    except SyntaxError:
        return "Error: Invalid syntax in mathematical expression"
    
# Second tool - Search tool

@tool
def search_internet(query):
    """Useful to search the internet
        about a a given topic and return relevant results"""
    try:
        top_result_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        # check if there is an organic key
        if 'organic' not in response.json():
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response.json()['organic']
            string = []
            for result in results[:top_result_to_return]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)
    except Exception:
        print("Exception occurred in search tool")


In [43]:
available_tools = [
    calculate,
    search_internet
]

functions = [convert_to_openai_function(f) for f in available_tools]

In [44]:
functions

[{'name': 'calculate',
  'description': 'Useful to perform any mathematical calculations,\n    like sum, minus, multiplication, division, etc.\n    The input to this tool should be a mathematical\n    expression, a couple examples are `200*7` or `5000/2*10`',
  'parameters': {'type': 'object',
   'properties': {'operation': {}},
   'required': ['operation']}},
 {'name': 'search_internet',
  'description': 'Useful to search the internet\n        about a a given topic and return relevant results',
  'parameters': {'type': 'object',
   'properties': {'query': {}},
   'required': ['query']}}]

In [45]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.environ["OPENAI_API_KEY"])

llm_with_tools = llm.bind(functions=functions)

In [46]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","you are a helpful AI assistant which helps users to resolve their queries"),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [47]:
chain = prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [48]:
agent = (
{
    "question": lambda x: x["question"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())


In [49]:
agent_executor = AgentExecutor(agent=agent, tools=available_tools, verbose=True)

In [50]:
agent_executor.invoke({"question": "Can you provide me the difference between the revenue of Google between Q1 2024 and Q2 2024?"})



> Entering new AgentExecutor chain...

Invoking: `search_internet` with `{'query': 'Google revenue Q1 2024 Q2 2024'}`


Title: [PDF] Alphabet Announces Second Quarter 2024 Results
Link: https://abc.xyz/assets/19/e4/3dc1d4d6439c81206370167db1bd/2024q2-alphabet-earnings-release.pdf
Snippet: Ruth Porat, President and Chief Investment Officer; CFO said: “We delivered revenues of $85 billion, up 14% year- on-year driven by Search as ...

-----------------
Title: Google Search Revenue Grows 14% In Q2 2024
Link: https://www.searchenginejournal.com/google-search-revenue-grows-14-in-q2-2024/522922/
Snippet: Google Search Revenue Grows 14% In Q2 2024 · Google Search revenue: $48.5 billion (up from $42.6 billion in Q2 2023) · Total Alphabet revenue: ...

-----------------
Title: Google had a massive quarter thanks to Search and AI - The Verge
Link: https://www.theverge.com/2024/7/23/24204618/google-search-ai-q2-2024-earnings-results
Snippet: Google parent company Alphabet made almost $85 billio

{'question': 'Can you provide me the difference between the revenue of Google between Q1 2024 and Q2 2024?',
 'output': 'In Q1 2024, Google (Alphabet Inc.) reported revenues of **$80.5 billion**. In Q2 2024, the revenue increased to **$85 billion**. \n\nTo find the difference in revenue between Q2 2024 and Q1 2024:\n\n\\[\n\\text{Difference} = \\text{Q2 Revenue} - \\text{Q1 Revenue} = 85 \\text{ billion} - 80.5 \\text{ billion} = 4.5 \\text{ billion}\n\\]\n\nThus, the difference in revenue between Q2 2024 and Q1 2024 is **$4.5 billion**.'}

In [51]:
agent_executor.invoke(
    {
        "question": """Hello there"""
    }
)



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'question': 'Hello there', 'output': 'Hello! How can I assist you today?'}